# Overview of OSC Debt Stock and Education Expenditure


In [1]:
# Import packages
import requests
import json
import pandas as pd
import wbgapi as wb
import datetime
import wbdata
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook" # use "pio.renderers" to see the default renderer


# Assess Sources using World Bank API

In [2]:
# Get all sources from the World Bank API
sources = requests.get("http://api.worldbank.org/v2/sources?per_page=100&format=json")
sourcesJSON = sources.json()
# print(sourcesJSON) # if you want to view the JSON response as is, remove the "#" at the beginning of the print command

# Parse through the response to see the source names and ID numbers.
for i in sourcesJSON[1]:
    if i["name"] == "International Debt Statistics":
        print("The source ID for International Debt Statistics is " + i["id"])
    else:
        pass
    #print(i["id"],i["name"]) # to see all the source names and IDs, remove the # at the beginning of this line

The source ID for International Debt Statistics is 6


In [3]:
# Requesting the indicators for the topic External Debt
indicators = requests.get("http://api.worldbank.org/v2/indicator?format=json&source=6")
indicatorsJSON = indicators.json()

In [4]:
# Print the total number of indicators
print("There are " + str(indicatorsJSON[0]["total"]) + " IDS indicators")

There are 572 IDS indicators


In [5]:
# Get all External Debt indicators, with a per_page parameter of 500.
indicators = requests.get("http://api.worldbank.org/v2/indicator?format=json&source=6&per_page=500")
indicatorsJSON = indicators.json()
# print(indicatorsJSON) # to view ALL of the indicators as is, remove the "#" at the beginning of the print command

# Parse through the response to see the Indicator IDs and Names
for i in indicatorsJSON[1]:
    IDSindicators = (i["id"],i["name"])
    print(IDSindicators) # to view the indicator ids and names, remove the "#" at the beginning of the print command

('BM.GSR.TOTL.CD', 'Imports of goods, services and primary income (BoP, current US$)')
('BN.CAB.XOKA.CD', 'Current account balance (BoP, current US$)')
('BX.GRT.EXTA.CD.DT', 'Grants, excluding technical cooperation (current US$)')
('BX.GRT.TECH.CD.DT', 'Technical cooperation grants (current US$)')
('BX.GSR.TOTL.CD', 'Exports of goods, services and primary income (BoP, current US$)')
('BX.KLT.DINV.CD.DT', 'Foreign direct investment, net inflows in reporting economy (DRS, current US$)')
('BX.KLT.DREM.CD.DT', 'Primary income on FDI (current US$)')
('BX.PEF.TOTL.CD.DT', 'Portfolio investment, equity (DRS, current US$)')
('BX.TRF.PWKR.CD.DT', 'Personal remittances, received (current US$)')
('DT.AMT.BLAT.CB.CD', 'CB, bilateral (AMT, current US$)')
('DT.AMT.BLAT.CD', 'PPG, bilateral (AMT, current US$)')
('DT.AMT.BLAT.GG.CD', 'GG, bilateral (AMT, current US$)')
('DT.AMT.BLAT.OPS.CD', 'OPS, bilateral (AMT, current US$)')
('DT.AMT.BLAT.PRVG.CD', 'PRVG, bilateral (AMT, current US$)')
('DT.AMT.BLA

In [7]:
# Use the indicator code to define the "indicator" variable
indicator = "DT.DOD.DLXF.CD"

# Parse through the response to get the "sourceNote" or definition for the desired indicator
for dict_entity in indicatorsJSON[1]:
    if dict_entity["id"] == indicator:
        print(dict_entity["sourceNote"])
    else:
        pass

Long-term debt is debt that has an original or extended maturity of more than one year. It has three components: public, publicly guaranteed, and private nonguaranteed debt. Data are in current U.S. dollars.


# Location Debtor and Creditor

In [8]:
# Requesting the locations
dlocations = requests.get("http://api.worldbank.org/v2/sources/6/country?per_page=300&format=JSON")
dlocationsJSON = dlocations.json()

# Parse through the response to see the location IDs and names
dlocations = dlocationsJSON["source"][0]["concept"][0]["variable"]
listLen = int(len(dlocations))

# Create dataframe with location values
df = pd.DataFrame(columns=["id", "value"])     
for i in range(0,listLen):
    code = dlocations[i]["id"]
    name = dlocations[i]["value"]
    # df = df.append({"id":code, "value":name}, ignore_index = True)
    rows_to_append = pd.DataFrame([{"id": code, "value": name}])
    df = pd.concat([df, rows_to_append], ignore_index=True)
dlocationsList = df

# See first few items in the dataframe
print(dlocationsList.head(n=10))

    id         value
0  AFG   Afghanistan
1  AGO        Angola
2  ALB       Albania
3  ARG     Argentina
4  ARM       Armenia
5  AZE    Azerbaijan
6  BDI       Burundi
7  BEN         Benin
8  BFA  Burkina Faso
9  BGD    Bangladesh


In [9]:
# Requesting the locations
clocations = requests.get("http://api.worldbank.org/v2/sources/6/counterpart-area?per_page=300&format=JSON")
clocationsJSON = clocations.json()

# Parse through the response to see the location IDs and names
clocations = clocationsJSON["source"][0]["concept"][0]["variable"]
listLen = int(len(clocations))

# Create dataframe with location values
df = pd.DataFrame(columns=["id", "value"])     
for i in range(0,listLen):
    code = clocations[i]["id"]
    name = clocations[i]["value"]
    # df = df.append({"id":code, "value":name}, ignore_index = True)
    rows_to_append = pd.DataFrame([{"id": code, "value": name}])
    df = pd.concat([df, rows_to_append], ignore_index=True)
clocationsList = df

# See first few items in the dataframe
print(clocationsList.head(n=10))

    id                  value
0  001                Austria
1  002                Belgium
2  003                Denmark
3  004                 France
4  005  Germany, Fed. Rep. of
5  006                  Italy
6  007            Netherlands
7  008                 Norway
8  009               Portugal
9  010                 Sweden


### Combining the debtor and creditor for a unified locations data

In [10]:
dlocations = requests.get("http://api.worldbank.org/v2/sources/6/country?per_page=300&format=JSON")
clocations = requests.get("http://api.worldbank.org/v2/sources/6/counterpart-area?per_page=300&format=JSON")

dlocationsJSON = dlocations.json()
clocationsJSON = clocations.json()

# Parse through the response to see the location IDs and names
dlocations = dlocationsJSON["source"][0]["concept"][0]["variable"]

# Parse through the response to see the location IDs and names
clocations = clocationsJSON["source"][0]["concept"][0]["variable"]

# Assuming dlocations and clocations are defined as lists of dictionaries
combined_locations = {
    "country_data": dlocations,
    "counterpart_data": clocations
}

# Dictionary to store the combined data
locations_dict = {}

# Process country_data
for entry in combined_locations["country_data"]:
    code = entry["id"]
    name = entry["value"]
    if name not in locations_dict:
        locations_dict[name] = {"locationName": name, "debtorCode": code, "creditorCode": ''}
    else:
        locations_dict[name]["debtorCode"] = code

# Process counterpart_data
for entry in combined_locations["counterpart_data"]:
    code = entry["id"]
    name = entry["value"]
    if name not in locations_dict:
        locations_dict[name] = {"locationName": name, "debtorCode": '', "creditorCode": code}
    else:
        locations_dict[name]["creditorCode"] = code

# Convert the dictionary to a DataFrame
df = pd.DataFrame(locations_dict.values())

# Sort the DataFrame by locationName
locationList = df.sort_values(by='locationName').reset_index(drop=True)

# Print the results
print(locationList.head(n=10))
print(len(locationList))
# Save the DataFrame to a CSV file
locationList.to_csv('LocationCodes.csv', index=False)




                      locationName debtorCode creditorCode
0                      Afghanistan        AFG          625
1       African Dev. Bank                              913
2       African Export-Import Bank                     815
3                          Albania        ALB          071
4                          Algeria        DZA          130
5                          Andorra                     025
6                           Angola        AGO          225
7                         Anguilla                     376
8                          Antigua                     377
9  Arab African International Bank                     922
323


# First look at external debt stocks

In [34]:
# Get all External Debt indicators, with a per_page parameter of 500.
indicators = requests.get("http://api.worldbank.org/v2/indicator?format=json&source=6&per_page=500")
indicatorsJSON = indicators.json()
# print(indicatorsJSON) # to view ALL of the indicators as is, remove the "#" at the beginning of the print command

# Parse through the response to see the Indicator IDs and Names
for i in indicatorsJSON[1]:
    # See metadata about the indicator
    if i["id"] == 'DT.DOD.DLXF.CD':
        # IDSindicators = (i["id"],i["name"])
        print(i) 

{'id': 'DT.DOD.DLXF.CD', 'name': 'External debt stocks, long-term (DOD, current US$)', 'unit': '', 'source': {'id': '6', 'value': 'International Debt Statistics'}, 'sourceNote': 'Long-term debt is debt that has an original or extended maturity of more than one year. It has three components: public, publicly guaranteed, and private nonguaranteed debt. Data are in current U.S. dollars.', 'sourceOrganization': 'World Bank, International Debt Statistics.', 'topics': [{'id': '20', 'value': 'External Debt'}]}


In [12]:

# Create dataframe with stock values
df_debt = pd.DataFrame(columns=["debt stock id", "debt stock name", "source note"])     
# Use the indicator code to define the "indicator" variable
indicator = "debt stock"

# Parse through the response to get the "sourceNote" or definition for the desired indicator
for dict_entity in indicatorsJSON[1]:
    if indicator in dict_entity["name"] or indicator in dict_entity["sourceNote"]:
          id =dict_entity["id"]
          name = dict_entity["name"]
          sourceNote = dict_entity["sourceNote"]
          rows_to_append = pd.DataFrame([{"debt stock id": id, "debt stock name": name, "source note": sourceNote}])
          df_debt = pd.concat([df_debt, rows_to_append], ignore_index=True)   
       
    else:
        pass

print(df_debt)

df_debt.to_csv('DebtStockIndicators.csv', index=False)

        debt stock id                                    debt stock name  \
0      DT.DFR.DPPG.CD        Debt forgiveness or reduction (current US$)   
1      DT.DOD.ALLC.CD  External debt stocks, concessional (DOD, curre...   
2      DT.DOD.ALLC.ZS       Concessional debt (% of total external debt)   
3      DT.DOD.DECB.CD  External debt stocks, central bank (PPG) (DOD,...   
4      DT.DOD.DECT.CD     External debt stocks, total (DOD, current US$)   
5   DT.DOD.DECT.CD.CG  Total change in external debt stocks (current ...   
6   DT.DOD.DECT.EX.ZS  External debt stocks (% of exports of goods, s...   
7   DT.DOD.DECT.GN.ZS                    External debt stocks (% of GNI)   
8      DT.DOD.DEGG.CD  External debt stocks, general government secto...   
9      DT.DOD.DEPS.CD  External debt stocks, public sector (PPG) (DOD...   
10     DT.DOD.DLXF.CD  External debt stocks, long-term (DOD, current ...   
11     DT.DOD.DOPS.CD  External debt stocks, other public sector (PPG...   
12     DT.DO

In [13]:
print(len(indicatorsJSON[1]))

500


In [14]:
longtermDebtCode = 'DT.DOD.DLXF.CD'
url = f"http://api.worldbank.org/v2/indicator/{longtermDebtCode}?format=json&source=6"
ExternalDebtIndicatorsLT = requests.get(url)
ExternalDebtIndicatorsLTJSON = ExternalDebtIndicatorsLT.json()


print(ExternalDebtIndicatorsLTJSON)


[{'page': 1, 'pages': 1, 'per_page': '50', 'total': 1}, [{'id': 'DT.DOD.DLXF.CD', 'name': 'External debt stocks, long-term (DOD, current US$)', 'unit': '', 'source': {'id': '6', 'value': 'International Debt Statistics'}, 'sourceNote': 'Long-term debt is debt that has an original or extended maturity of more than one year. It has three components: public, publicly guaranteed, and private nonguaranteed debt. Data are in current U.S. dollars.', 'sourceOrganization': 'World Bank, International Debt Statistics.', 'topics': [{'id': '20', 'value': 'External Debt'}]}]]


### Access country details - basic world bank api

In [15]:
country_code = 'ETH'
url = f"http://api.worldbank.org/v2/country/{country_code}?format=JSON"
ETHInfo = requests.get(url)
ETHInfoJSON = ETHInfo.json()

print(ETHInfoJSON[1])

[{'id': 'ETH', 'iso2Code': 'ET', 'name': 'Ethiopia', 'region': {'id': 'SSF', 'iso2code': 'ZG', 'value': 'Sub-Saharan Africa '}, 'adminregion': {'id': 'SSA', 'iso2code': 'ZF', 'value': 'Sub-Saharan Africa (excluding high income)'}, 'incomeLevel': {'id': 'LIC', 'iso2code': 'XM', 'value': 'Low income'}, 'lendingType': {'id': 'IDX', 'iso2code': 'XI', 'value': 'IDA'}, 'capitalCity': 'Addis Ababa', 'longitude': '38.7468', 'latitude': '9.02274'}]


# External Debt | approach using wbgapi


In [16]:
# basic source info
wb.source.info()

id,name,code,concepts,lastupdated
1,Doing Business,DBS,3,2021-08-18
2,World Development Indicators,WDI,3,2024-06-28
3,Worldwide Governance Indicators,WGI,3,2023-09-29
5,Subnational Malnutrition Database,SNM,3,2016-03-21
6,International Debt Statistics,IDS,4,2024-02-29
11,Africa Development Indicators,ADI,3,2013-02-22
12,Education Statistics,EDS,3,2024-06-25
13,Enterprise Surveys,ESY,3,2022-03-25
14,Gender Statistics,GDS,3,2024-07-01
15,Global Economic Monitor,GEM,3,2024-07-10


In [ ]:
# indicators for default source (2)
wb.series.info()

In [20]:
# swtich to IDS (source 6)
wb.db = 6

In [32]:
indicator_code = "DT.DOD.DLXF.CD"
externalDebtStockLT = wb.series.info(indicator_code)
print(externalDebtStockLT)

id              value
--------------  --------------------------------------------------
DT.DOD.DLXF.CD  External debt stocks, long-term (DOD, current US$)
                1 elements


In [22]:
# Access External debt stocks, long-term (DOD, current US$)  | DT.DOD.DLXF.CD

locationSelection = ["ECA","SSA","SAS","LAC","MNA","EAP"]
time_range = range(2010, 2021)

# Query 1: Selected countries
print("Query: Selected countries")
print("\n" + "-"*50 + "\n")
try:
    EXD = wb.data.DataFrame(indicator_code, locationSelection,time=time_range, skipBlanks=True, columns='series').reset_index()
    EXD.rename(columns={indicator_code: "ExternalDebtStock"}, inplace=True)
    print(EXD.head(14))
except Exception as e:
    print(f"An error occurred: {e}")

print("\n" + "-"*50 + "\n")



Query: Selected countries

--------------------------------------------------

   counterpart_area economy    time  ExternalDebtStock
0               001     EAP  YR2010       1.759242e+09
1               001     EAP  YR2011       1.603839e+09
2               001     EAP  YR2012       1.508823e+09
3               001     EAP  YR2013       1.601138e+09
4               001     EAP  YR2014       1.294395e+09
5               001     EAP  YR2015       1.137716e+09
6               001     EAP  YR2016       1.019714e+09
7               001     EAP  YR2017       1.138541e+09
8               001     EAP  YR2018       9.871938e+08
9               001     EAP  YR2019       9.478912e+08
10              001     EAP  YR2020       9.962994e+08
11              001     ECA  YR2010       5.351414e+08
12              001     ECA  YR2011       5.959493e+08
13              001     ECA  YR2012       6.160618e+08

--------------------------------------------------



In [23]:
# Reshape the data
EXDreshaped = pd.DataFrame(EXD.to_records())
# EXDreshaped.to_csv("ExternalDebtStock20102020.csv")
print(EXDreshaped.head())

   index counterpart_area economy    time  ExternalDebtStock
0      0              001     EAP  YR2010       1.759242e+09
1      1              001     EAP  YR2011       1.603839e+09
2      2              001     EAP  YR2012       1.508823e+09
3      3              001     EAP  YR2013       1.601138e+09
4      4              001     EAP  YR2014       1.294395e+09


In [24]:
# Creating a function that will change units to billions and round to 0 decimal places
def formatNum(x):
    return round(x/1000000000)
    

# Running the function on the desired data column
EXDreshaped.ExternalDebtStock = formatNum(EXDreshaped.ExternalDebtStock)
print(EXDreshaped.head())

   index counterpart_area economy    time  ExternalDebtStock
0      0              001     EAP  YR2010                2.0
1      1              001     EAP  YR2011                2.0
2      2              001     EAP  YR2012                2.0
3      3              001     EAP  YR2013                2.0
4      4              001     EAP  YR2014                1.0


In [25]:
# Renaming column headers
EXDclean = EXDreshaped.rename(index=str, columns={
    "time":"Year",
    "economy":"Region",
    indicator_code: "ExternalDebtStock"
})

In [26]:
# Remove the "(excluding high income)" from each of the region names
EXDclean["Region"] = EXDclean["Region"].str.replace("excluding high income","").str.replace(")","").str.replace("(","")

In [27]:
print(EXDclean.head())

   index counterpart_area Region    Year  ExternalDebtStock
0      0              001    EAP  YR2010                2.0
1      1              001    EAP  YR2011                2.0
2      2              001    EAP  YR2012                2.0
3      3              001    EAP  YR2013                2.0
4      4              001    EAP  YR2014                1.0


In [24]:
   
source = EXDclean

# Creating the chart
chart = px.line(EXDclean, 
                x="Year",
                y="ExternalDebtStock",
                color="Region",
                title="Regional Long-term External Debt Stock (excluding High-Income countries)(USD billion)",
                )
                
                
chart.update_layout(
                plot_bgcolor="white")

# Displaying the chart
chart 

# Education

### Per percentage of total Expenditure

In [85]:
wb.db = 2
edExpIndicators = wb.series.info()
print(edExpIndicators)





id                         value
-------------------------  ---------------------------------------------------------------------------------------------------------------------------------------------
AG.AGR.TRAC.NO             Agricultural machinery, tractors
AG.CON.FERT.PT.ZS          Fertilizer consumption (% of fertilizer production)
AG.CON.FERT.ZS             Fertilizer consumption (kilograms per hectare of arable land)
AG.LND.AGRI.K2             Agricultural land (sq. km)
AG.LND.AGRI.ZS             Agricultural land (% of land area)
AG.LND.ARBL.HA             Arable land (hectares)
AG.LND.ARBL.HA.PC          Arable land (hectares per person)
AG.LND.ARBL.ZS             Arable land (% of land area)
AG.LND.CREL.HA             Land under cereal production (hectares)
AG.LND.CROP.ZS             Permanent cropland (% of land area)
AG.LND.EL5M.RU.K2          Rural land area where elevation is below 5 meters (sq. km)
AG.LND.EL5M.RU.ZS          Rural land area where elevation is below 5 m

In [87]:
# indicator_code = ["SE.XPD.TOTL.GB.ZS","SE.XPD.TOTL.GD.ZS"]
indicator_code = "SE.XPD.TOTL.GB.ZS"
locationSelection = ["ECA","SSA","SAS","LAC","MNA","EAP"]
time_range = range(2010, 2021)

# Query 1: All countries
print("Query: Selected countries")
try:
    EDE = wb.data.DataFrame(indicator_code, locationSelection,time=time_range, skipBlanks=True, columns='series').reset_index()
    EDE.rename(columns={indicator_code: "EducationExpenditureTotal"}, inplace=True)
    # EXD.sort_values(by='economy', ascending=True, inplace=True)
    print(EDE.head(14))
except Exception as e:
    print(f"An error occurred: {e}")

print("\n" + "-"*50 + "\n")

Query: Selected countries
   economy    time  EducationExpenditureTotal
0      EAP  YR2010                  15.185876
1      EAP  YR2011                  13.488172
2      EAP  YR2012                  14.407168
3      EAP  YR2013                  14.005000
4      EAP  YR2014                  15.058273
5      EAP  YR2015                  15.167500
6      EAP  YR2016                  15.115000
7      EAP  YR2017                  13.814648
8      EAP  YR2018                  15.309965
9      EAP  YR2019                  14.499084
10     EAP  YR2020                  13.378041
11     ECA  YR2011                  12.952096
12     ECA  YR2012                  14.007706
13     ECA  YR2013                  12.392527

--------------------------------------------------



In [88]:
# Reshape the data
EDEreshaped = pd.DataFrame(EDE.to_records())
EDEreshaped.to_csv("EducationExpenditurePerExpenditure.csv")

# Renaming column headers
EDEclean = EDEreshaped.rename(index=str, columns={
    "time":"Year",
    "economy":"Region",
    indicator_code: "EducationExpenditureTotal"
})

In [89]:
   
source = EDEclean

# Creating the chart
chart = px.line(EDEclean, 
                x="Year",
                y="EducationExpenditureTotal",
                color="Region",
                title="Regional Education Expenditure Total (% of total government expenditure) (excluding High-Income countries)(USD billion)",
                )
                
                
chart.update_layout(
                plot_bgcolor="white")

# Displaying the chart
chart 

### Per percentage of GDP

In [90]:
# indicator_code = ["SE.XPD.TOTL.GB.ZS","SE.XPD.TOTL.GD.ZS"]
indicator_code = "SE.XPD.TOTL.GD.ZS"
locationSelection = ["ECA","SSA","SAS","LAC","MNA","EAP"]
time_range = range(2010, 2021)

# Query 1: All countries
print("Query: Selected countries")
try:
    EDE = wb.data.DataFrame(indicator_code, locationSelection,time=time_range, skipBlanks=True, columns='series').reset_index()
    EDE.rename(columns={indicator_code: "EducationExpenditureTotal"}, inplace=True)
    # EXD.sort_values(by='economy', ascending=True, inplace=True)
    print(EDE.head(14))
except Exception as e:
    print(f"An error occurred: {e}")

print("\n" + "-"*50 + "\n")

Query: Selected countries
   economy    time  EducationExpenditureTotal
0      EAP  YR2010                   3.754642
1      EAP  YR2011                   3.651098
2      EAP  YR2012                   3.770500
3      EAP  YR2013                   3.767000
4      EAP  YR2014                   3.540000
5      EAP  YR2015                   3.583600
6      EAP  YR2016                   3.569971
7      EAP  YR2017                   3.442000
8      EAP  YR2018                   3.246685
9      EAP  YR2019                   3.085000
10     EAP  YR2020                   3.531387
11     ECA  YR2010                   3.843973
12     ECA  YR2011                   4.221520
13     ECA  YR2012                   3.959858

--------------------------------------------------



In [91]:
# Reshape the data
EDEreshaped = pd.DataFrame(EDE.to_records())
EDEreshaped.to_csv("EducationExpenditurePerGDP.csv")

# Renaming column headers
EDEclean = EDEreshaped.rename(index=str, columns={
    "time":"Year",
    "economy":"Region",
    indicator_code: "EducationExpenditureTotal"
})

In [92]:
   
source = EDEclean

# Creating the chart
chart = px.line(EDEclean, 
                x="Year",
                y="EducationExpenditureTotal",
                color="Region",
                title="Regional Education Expenditure Total (% of GDP) (excluding High-Income countries)(USD billion)",
                )
                
                
chart.update_layout(
                plot_bgcolor="white")

# Displaying the chart
chart 